# Tutorial from [Geeks for Geeks](https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/)

- using conda environment named 'data_analysis' (Python 3.12.3)

In [ ]:
# run the command 'python -m textblob.download_corpora' to get structured text to work with

In [5]:
# import libraries
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import json

In [25]:
import tweepy.errors


class TwitterClient(object):
    '''
    This is a twitter class for sentiment analysis.
    '''
    def __init__(self):
        ''' 
        initialization / constructor
        '''
        # get keys and tokens from the Twitter Dev Console
        consumer_key = 'OIZCfcPsuGm56nUnyT7siCcSu'
        consumer_secret = 'cyCPUrMbDJYcu95RfA80DMeQfhgRzU7ssvVnR9g2LDBIxrr1oM'
        access_token = '1259988784201990149-DBcQY3pP7VcXM7QU0q84Z17Dsf3aoB'
        access_token_secret = 'rxRLvjU4OCus8sfQCbHkMXfWBk5P8hQTKYhSSUa7ZgacY'

        # attempt authentication
        try:
            # create handler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed.")
        
    def clean_tweet(self, tweet):
        '''
        Clean tweet text by removing links and special characters.
        '''
        return ' '.join(re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
    
    def get_tweet_sentiment(self, tweet):
        '''
        Classify sentiment of passed tweet using textblob's sentiment method.
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
        
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []

        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search_tweets(q = query, count = count)

            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}

                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                    else:
                        tweets.append(parsed_tweet)
            
            # return parsed tweets
            return tweets
        except tweepy.errors.TweepyException as e:
            # print error (if any)
            print("Error : " + str(e))

In [26]:
def get_analysis(query, count = 200, saving = True, file_name='example.json'):
    # create object of TwitterClient class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query=query, count = count)
    if saving:
        with open(file_name, 'w') as f:
            json.dump(tweets, f)
    return tweets

In [27]:
tweets = get_analysis('Nursing Homes', 200, True, 'Nursing_Homes.json')

Error : 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product


In [28]:
import requests

# define your bearer token
bearer_token = 'AAAAAAAAAAAAAAAAAAAAACwqtwEAAAAAJXIQ%2FM5EsAZf75YwKs9Qoskil9o%'

# define the endpoint URL
endpoint_url = 'https://api.twitter.com/2/tweets/search/recent'

# define the query parameters
query_params = {
    'query': 'your_search_query',
    'max_results': 10  # Adjust as needed
}

# define headers
headers = {
    'Authorization': f'Bearer {bearer_token}'
}

# make the request
response = requests.get(endpoint_url, params=query_params, headers=headers)

# check if the request was successful
if response.status_code == 200:
    # parse the JSON response
    data = response.json()
    
    # access the tweets
    tweets = data['data']
    
    # process the tweets as needed
    for tweet in tweets:
        print(tweet['text'])
else:
    # print the error message
    print(f'Error: {response.status_code}')
    print(response.text)


Error: 500



## Twitter API Status
- Twitter has ended free access to its API - now, there is a limited [free-tier](https://devcommunity.x.com/t/is-free-access-to-the-twitter-api-officially-no-longer-available/200822) available which is not very useful.  